In [19]:
import zipfile
import os
import numpy as np
import pathlib
import pandas as pd
from math import ceil
import tensorflow as tf
import numpy as np
import IPython.display as display
import keras
from keras import backend as K
from matplotlib import pyplot as plt
from keras.utils import to_categorical
from keras_preprocessing.image import ImageDataGenerator

print(tf.__version__)

root_path = './'
dataset_path = os.path.join(root_path, 'faces-data-new')

print(dataset_path)

models_path = os.path.join(root_path, 'saved_models_new')
if not os.path.exists(models_path):
    os.mkdir(models_path)


tgt_pth = os.path.join(root_path, 'visualize_dcgan-v1')

if not os.path.exists(tgt_pth):
    os.mkdir(tgt_pth)



BATCH_SIZE = 32
img_rows, img_cols, img_depth = (196, 196, 3)


def rescaleFn(img):
    return img / 127.5 - 1


datagen=ImageDataGenerator(preprocessing_function=rescaleFn)


train_generator = datagen.flow_from_directory(
    directory=dataset_path,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=True,
)


STEP_SIZE=train_generator.n//train_generator.batch_size


X = next(train_generator)
print(X.shape)

1.13.1
./faces-data-new
Found 7864 images belonging to 1 classes.
(32, 256, 256, 3)


In [14]:
img_shape = (196, 196, 3)
img_len = np.prod(img_shape)
latent_dim = img_len
noise_len = 100
input_dim = noise_len
ngf = 64
ndf = 64


def build_discriminator():
    return keras.Sequential([
        # conv block 1
        keras.layers.Conv2D(
            filters=ndf,
            kernel_size=(4, 4),
            strides=2,
            input_shape=img_shape,
            use_bias=False
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 2
        keras.layers.Conv2D(
            filters=ndf * 2,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 3
        keras.layers.Conv2D(
            filters=ndf * 4,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 4
        keras.layers.Conv2D(
            filters=ndf * 8,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 5
        keras.layers.Conv2D(
            filters=1,
            kernel_size=(9, 9),
            strides=2,
            use_bias=False
        ),
        
        #output
        keras.layers.Activation(tf.nn.sigmoid)
    ])


def build_generator():
    
    return keras.Sequential([
        # reshape 1d to 3d
        keras.layers.Reshape((10, 10, 1), input_shape=(input_dim,)),
        
        # transpose conv block 1
        keras.layers.Conv2DTranspose(
            filters=ngf * 8,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 2
        keras.layers.Conv2DTranspose(
            filters=ngf * 4,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 3
        keras.layers.Conv2DTranspose(
            filters=ngf * 2,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 4
        keras.layers.Conv2DTranspose(
            filters=ngf,
            kernel_size=(4, 4),
            strides=1,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 5
        keras.layers.Conv2DTranspose(
            filters=3,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False
        ),
        
        # output
        keras.layers.Activation(tf.nn.tanh)
    ])


discriminator = build_discriminator()
generator = build_generator()

generator.summary()
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_8 (Reshape)          (None, 10, 10, 1)         0         
_________________________________________________________________
conv2d_transpose_30 (Conv2DT (None, 22, 22, 512)       8192      
_________________________________________________________________
batch_normalization_47 (Batc (None, 22, 22, 512)       2048      
_________________________________________________________________
activation_70 (Activation)   (None, 22, 22, 512)       0         
_________________________________________________________________
conv2d_transpose_31 (Conv2DT (None, 46, 46, 256)       2097152   
_________________________________________________________________
batch_normalization_48 (Batc (None, 46, 46, 256)       1024      
_________________________________________________________________
activation_71 (Activation)   (None, 46, 46, 256)       0         
__________

In [15]:
GLR = 0.0002  # generator
DLR = 0.0002  # discriminator


discriminator.compile(
    optimizer=keras.optimizers.Adam(DLR, 0.5),
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

# noise
x = keras.layers.Input(shape=(noise_len,))

# freeze discriminator
discriminator.trainable = False

# output
z = generator(x)
out = discriminator(z)

# GAN
gan = keras.models.Model(inputs=x, outputs=out)

gan.compile(
    optimizer=keras.optimizers.Adam(GLR , 0.5),
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy'])

In [16]:
def visualizeGAN(e, z_real, z_fake):

    fig, axes = plt.subplots(4, 4, figsize=(20, 18))

    r_real = 0
    r_fake = 0
    for row, axe in enumerate(axes):
        for col, cell in enumerate(axe):
            if row % 2 == 0:
                cell.imshow(
                    np.squeeze(
                        0.5 * z_real[r_real * 4 + col] + 0.5,
                        axis=-1
                    ),
                    cmap='gray'
                )
            else:
                cell.imshow(
                    np.squeeze(
                        0.5 * z_fake[r_fake * 4 + col] + 0.5,
                        axis=-1
                    ),
                    cmap='gray'
                )

            cell.axis("off")

        if row % 2 == 0:
            r_real += 1
        else:
            r_fake += 1

    plt.axis("off")
    plt.tight_layout()

    fig.savefig(os.path.join(tgt_pth, '{}.jpg'.format(str(e).zfill(3))))
    
    plt.close()

In [ ]:
EPOCHS = 101
STEPS = 1


train_loss_g = []
train_loss_d = []

train_acc_g = []
train_acc_d = []


# to be visualized
vis_noise = np.random.normal(size=(16, noise_len,))
print(vis_noise.shape)

# fake and real label
y_fake = np.zeros(size=(BATCH_SIZE,))
y_real = np.ones(size=(BATCH_SIZE,))


# epochs
for e in range(EPOCHS):

    #batches
    loss = []
    acc = []

    for p in range(STEPS):
        
        x, z_fake, y_fake, z_real, y_real = next(disc_itr)
        z = np.random.normal(size=(BATCH_SIZE, noise_len))
        x_fake = generator.predict(z)
        x_real = next(train_generator)

        # train
        loss_2, acc_2 = discriminator.train_on_batch(x_real, y_real)
        loss_1, acc_1 = discriminator.train_on_batch(x_fake, y_fake)

        batch_loss = 0.5 * (loss_1 + loss_2)
        batch_acc = 0.5 * (acc_1 + acc_2)

        loss.append(batch_loss)
        acc.append(batch_acc)

    train_loss_d.append(np.mean(np.array(loss)))
    train_acc_d.append(np.mean(np.array(acc)))

    #batches
    loss = []
    acc = []

    for p in range(STEPS):

        z = np.random.normal(size=(BATCH_SIZE, noise_len))

        # train
        loss_1, acc_1 = gan.train_on_batch(z, y_true)

        loss.append(loss_1)
        acc.append(acc_1)

    train_loss_g.append(np.mean(np.array(loss)))
    train_acc_g.append(np.mean(np.array(acc)))


    print("E: {}, [D ACC: %{:.2f}], [D LOSS: {:.2f}], [G ACC: %{:.2f}], [G LOSS: {:.2f}]".format(
          e,
          train_acc_d[-1] * 100,
          train_loss_d[-1] * 100,
          train_acc_g[-1] * 100,
          train_loss_g[-1] * 100
      ))

    if e % 25 == 0:
        ## visualize results
        viz_fake = generator.predict(vis_noise)
        visualizeGAN(e, viz_real, viz_fake)
        
        ## save model
        pth = os.path.join(models_path, 'gan.h5')
        gan.save(pth)

        pth = os.path.join(models_path, 'generator-{}-{}-{}.h5'.format(e, train_loss_g[-1], train_acc_g[-1]))
        generator.save(pth)

        pth = os.path.join(models_path, 'discriminator.h5')
        discriminator.save(pth)

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_loss_g, label="Generator Loss");
plt.plot(train_loss_d, label="Discriminator Loss");
plt.legend();

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_acc_g, label="Generator Accuracy");
plt.plot(train_acc_d, label="Discriminator Accuracy");
plt.legend();